<a href="https://colab.research.google.com/github/balavishnu266/Leveraging_LLM_for_Text_Summaries/blob/main/LLM_Preprocess_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the necessary library to mount Google Drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to your file in Google Drive
file_path = '/content/drive/My Drive/WiSe22-Bar01.docx'

In [ ]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 1.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from docx import Document

# Read the docx file
doc = Document(file_path)

# Extract text from the document
text = []
for paragraph in doc.paragraphs:
    text.append(paragraph.text)

# Convert the text to a pandas DataFrame
df = pd.DataFrame(text, columns=['Text'])

# Print the contents of the file
print(df)


                                                 Text
0                  Interviewtranskript „WiSe22-Bar01“
1                         Interviewcode: WiSe22-Bar01
2        Datum und Uhrzeit des Interviews: 28.01.2023
3                  Dauer des Interviews (Minuten): 92
4                     Interview durchgeführt von: Bar
5                    Interview transkribiert von: Bar
6                                                    
7   Synonym der interviewten Person (Kürzel): Mari...
8        Geschlecht der interviewten Person: weiblich
9                   Alter der interviewten Person: 20
10  Studiengang der interviewten Person: Lehramtss...
11        Fachsemesterzahl der interviewten Person: 3
12                                                   
13  Transkriptionsregeln: Transkript verbatim mit ...
14  Sonstige Anmerkungen und Beobachtungen zur Int...
15  Synonyme: Im Folgenden werden der Interviewer ...
16                                                   
17  I: So, ich habe jetzt di

In [ ]:
#Preprocessing with NLP
df['Text'] = df['Text'].str.lower()

In [ ]:
import re

# Remove special characters, punctuation, and numbers using regular expressions
df['Text'] = df['Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))  # Remove special characters

In [ ]:
#Tokenization
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize

df['Tokens'] = df['Text'].apply(word_tokenize)

In [ ]:
#Removing Stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('german'))
df['Tokens'] = df['Tokens'].apply(lambda x: [word for word in x if word not in stop_words])


In [ ]:
#Stemming
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('german')
df['Stemmed'] = df['Tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

In [ ]:
#Joining the tokens back to sentences
df['Processed_Text'] = df['Stemmed'].apply(lambda x: ' '.join(x))

In [ ]:
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 30.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [ ]:
from transformers import pipeline

# Load the NER pipeline with the German BERT model
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-german")

# Iterate over each row in the DataFrame and perform NER
for index, row in df.iterrows():
    text = row['Text']
    ner_results = ner_pipeline(text)
    print("Entities in row", index)
    for entity in ner_results:
        print(entity["word"], entity["entity"])
    print()


OSError: dbmdz/bert-large-german-cased is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`